In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import selenium
import urllib.request

In [2]:
df = pd.read_csv("C:/worksplaces/workspace_project/project_visual/restaurant_data/data/restaurant.csv", encoding="utf-8", index_col=0)
df

,BPLCNM,TRDSTATENM,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,X,Y
0,북악파크부페,영업/정상,110846.0,서울특별시 종로구 평창동 110-1번지,서울특별시 종로구 평창길 352 (평창동),197993.205677,456584.627065
1,시민식당 2호점,영업/정상,110290.0,서울특별시 종로구 인사동 98번지,"서울특별시 종로구 종로11길 10, 1,2층 (인사동)",198681.571899,452050.274345
2,일번우동,영업/정상,110126.0,서울특별시 종로구 종로6가 129-1번지,서울특별시 종로구 종로41길 10 (종로6가),200533.453584,452156.900283
3,서촌소국밥,영업/정상,110043.0,서울특별시 종로구 통인동 27-2,서울특별시 종로구 필운대로6길 17-1 (통인동),197250.385438,453135.451266
4,빽다방 동대문역점,영업/정상,110843.0,서울특별시 종로구 창신동 696-1,서울특별시 종로구 종로 289 (창신동),200811.925354,452162.569488
...,...,...,...,...,...,...,...
124905,20bucks,영업/정상,130831.0,서울특별시 동대문구 이문동 316-2 지하1층,"서울특별시 동대문구 휘경로2길 19, 지하1층 (이문동)",205294.170617,454765.261112
124906,DNJ,영업/정상,139230.0,서울특별시 노원구 하계동 256-11 보스턴 산부인과,"서울특별시 노원구 노원로 244, 보스턴 산부인과 1층 (하계동)",206490.594308,459688.060943
124907,희래등반점,영업/정상,110811.0,서울특별시 종로구 명륜3가 1-14,서울특별시 종로구 명륜길 31 (명륜3가),199506.183606,454029.397330
124908,별달맛집,영업/정상,135894.0,서울특별시 강남구 신사동 609-1 지하1층,"서울특별시 강남구 압구정로30길 23 (신사동,지하1층)",202534.550000,447079.205000


In [3]:
df.isnull().sum()

BPLCNM            1
TRDSTATENM        0
SITEPOSTNO       80
SITEWHLADDR      80
RDNWHLADDR     1660
X              1880
Y              1880
dtype: int64

### Null 값 처리 어떻게 할지 고민해보자
- BPLCNM 없는 로우의 주소를 검색해서 입력해주자
- SITEPOSTNO 칼럼은 필요없음 
- SITEWHLADDR에서 없는 데이터는 RDNWHLADDR 데이터에 있을까 (또는 반대)
- X와 Y에 없는 데이터의 위경도 가져오는 법?
- X와 Y가 없는 데이터의 주소는 다 있는 건가?

In [4]:
nm_null = df[df['BPLCNM'].isnull()].index

df.iloc[nm_null, 0] = ('엔에이')
df.iloc[nm_null]

,BPLCNM,TRDSTATENM,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,X,Y
10449,엔에이,영업/정상,100848.0,서울특별시 중구 을지로4가 35번지 2층,"서울특별시 중구 창경궁로5길 27, 2층 (을지로4가)",199614.974362,451625.623159


In [5]:
df[(df['SITEWHLADDR'].isnull()) & (df['RDNWHLADDR'].isnull())]

,BPLCNM,TRDSTATENM,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,X,Y


> df['SITEWHLADDR'] 이 null이면서 df['RDNWHLADDR']이 null 인 값은 없음

In [6]:
addr_null = df[df['SITEWHLADDR'].isnull()].index

In [7]:
df['SITEWHLADDR'] = np.where(pd.notnull(df['SITEWHLADDR']) == True, df['SITEWHLADDR'], df['RDNWHLADDR'])

In [8]:
df.isnull().sum()

BPLCNM            0
TRDSTATENM        0
SITEPOSTNO       80
SITEWHLADDR       0
RDNWHLADDR     1660
X              1880
Y              1880
dtype: int64

In [9]:
df['BPLCNM'].replace('메가엠지씨커피', '메가커피', inplace=True)

In [10]:
df[df['BPLCNM'].str.contains("메가커피")]

,BPLCNM,TRDSTATENM,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,X,Y
8269,메가커피 명보사거리점,영업/정상,100300.0,서울특별시 중구 초동 32-6 (지상1층),"서울특별시 중구 마른내로 42-1 (초동,(지상1층))",199280.740644,451320.659651
26114,메가커피,영업/정상,130810.0,서울특별시 동대문구 신설동 84-5번지 1층,"서울특별시 동대문구 왕산로 3, 1층 (신설동)",202026.286929,452656.319191
27638,메가커피 장안점,영업/정상,130840.0,서울특별시 동대문구 장안동 366-1번지,"서울특별시 동대문구 장한로 116, 1층 (장안동)",206208.568083,452051.892512
33720,메가커피 북한산 정릉점,영업/정상,136878.0,서울특별시 성북구 정릉동 780,"서울특별시 성북구 보국문로 167 (정릉동, 산장상가8~9호)",200125.893637,457155.050338
40634,메가커피창동신창점,영업/정상,132916.0,서울특별시 도봉구 창동 501-20,서울특별시 도봉구 우이천로4길 31 (창동),203556.254399,459086.493367
41249,메가커피 신도봉시장점,영업/정상,132846.0,서울특별시 도봉구 방학동 653-18 1층,"서울특별시 도봉구 도봉로 727, 1층 (방학동)",203743.863745,462995.468546
41864,메가커피(신창시장점),영업/정상,132917.0,서울특별시 도봉구 창동 569-85번지,서울특별시 도봉구 덕릉로 238 (창동),203344.146254,459623.499162
51894,메가커피 이수태평점,영업/정상,156816.0,서울특별시 동작구 사당동 132-8 1층,"서울특별시 동작구 동작대로27길 20, 1층 (사당동)",198240.435377,442826.714216
99402,메가커피홍대점,영업/정상,121893.0,서울특별시 마포구 서교동 369-5 1층,"서울특별시 마포구 잔다리로6길 39 (서교동,1층)",192980.814895,450022.720414
101249,메가커피 양천신정중앙점,영업/정상,158861.0,서울특별시 양천구 신정동 1025-9 1층,"서울특별시 양천구 은행정로 17, 1층 (신정동)",187525.097764,446562.375457


In [11]:
df['BPLCNM'].replace('꾸쁘(Coupe)', '라꾸쁘', inplace=True)
df['BPLCNM'].replace('메가커피 우장산점', '메가커피 우장산역점', inplace=True)


### 1

In [12]:
craw_data = df[['BPLCNM', 'SITEWHLADDR', 'RDNWHLADDR']]

In [13]:
craw_data['Gu'] = craw_data.SITEWHLADDR.str.split(' ').str[1]
craw_data['Dong'] = craw_data.SITEWHLADDR.str.split(' ').str[2]

C:\Users\wogml\AppData\Local\Temp/ipykernel_19932/739233450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  craw_data['Gu'] = craw_data.SITEWHLADDR.str.split(' ').str[1]
C:\Users\wogml\AppData\Local\Temp/ipykernel_19932/739233450.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  craw_data['Dong'] = craw_data.SITEWHLADDR.str.split(' ').str[2]


In [14]:
craw_data = craw_data[['Gu', 'Dong', 'BPLCNM']]
address = craw_data['Gu'] + " " + craw_data['Dong'] + " " + craw_data['BPLCNM']

In [15]:
craw_data

,Gu,Dong,BPLCNM
0,종로구,평창동,북악파크부페
1,종로구,인사동,시민식당 2호점
2,종로구,종로6가,일번우동
3,종로구,통인동,서촌소국밥
4,종로구,창신동,빽다방 동대문역점
...,...,...,...
124905,동대문구,이문동,20bucks
124906,노원구,하계동,DNJ
124907,종로구,명륜3가,희래등반점
124908,강남구,신사동,별달맛집


In [16]:
craw_list = list(craw_data['BPLCNM'].unique())

In [17]:
address_list = list(address)
#print(address_list)

In [18]:
address_list

['종로구 평창동 북악파크부페',
 '종로구 인사동 시민식당 2호점',
 '종로구 종로6가 일번우동',
 '종로구 통인동 서촌소국밥',
 '종로구 창신동 빽다방 동대문역점',
 '종로구 묘동 팔도식당',
 '종로구 교북동 군산집',
 '종로구 숭인동 올리브떡볶이(동묘점)',
 '종로구 공평동 소문',
 '종로구 소격동 주식회사 가배도 삼청',
 '종로구 숭인동 강원맛코다리',
 '종로구 안국동 이화돈까스',
 '종로구 통인동 학아재',
 '종로구 충신동 기생',
 '종로구 종로5가 마이기어 종로점',
 '종로구 무악동 상구네백반',
 '종로구 효제동 스쿨푸드 딜리버리 종로',
 '종로구 연건동 서울대병원 델라코트',
 '종로구 연건동 서울대병원 가화원',
 '종로구 숭인동 홈푸드 스토리',
 '종로구 공평동 한옥집 김치찜',
 '종로구 숭인동 맷돌로만 숭인점',
 '종로구 창신동 로열네팔레스토랑&바르Ⅱ',
 '종로구 수송동 장수통닭',
 '종로구 사직동 호센',
 '종로구 신문로1가 뉴오리진 광화문점',
 '종로구 혜화동 나인티나인',
 '종로구 도렴동 오쵸',
 '종로구 숭인동 뜰채',
 '종로구 사직동 아르크',
 '종로구 돈의동 행운돈까스지기',
 '종로구 사직동 미요시',
 '종로구 익선동 아담집',
 '종로구 구기동 스튜디오 이여로',
 '종로구 계동 계동만담',
 '종로구 평창동 백석동주',
 '종로구 통인동 퀴진 라끌레(Cuisine La cle)',
 '종로구 관철동 도쿄술집 종로관철점',
 '종로구 평창동 두레유',
 '종로구 내수동 연정',
 '종로구 낙원동 어심',
 '종로구 세종로 애드',
 '종로구 당주동 국시방',
 '종로구 충신동 우미옥',
 '종로구 신문로2가 시인',
 '종로구 종로3가 가르텐 비어',
 '종로구 명륜3가 명륜 쭈꾸미',
 '종로구 명륜4가 파닭',
 '종로구 명륜3가 곰돌이',
 '종로구 동숭동 홍콩반점 0410',
 '종로구 운니동 미성',
 '종로구 연건동 효제',
 '종로구 명륜4가 뚜레쥬르',
 '종로구 

In [19]:
row_data = {'address' : address_list}
rest_add = pd.DataFrame(row_data)

In [20]:
rest_add

,address
0,종로구 평창동 북악파크부페
1,종로구 인사동 시민식당 2호점
2,종로구 종로6가 일번우동
3,종로구 통인동 서촌소국밥
4,종로구 창신동 빽다방 동대문역점
...,...
124905,동대문구 이문동 20bucks
124906,노원구 하계동 DNJ
124907,종로구 명륜3가 희래등반점
124908,강남구 신사동 별달맛집


# 크롤링


selenium<br>
https://0433.tistory.com/41

In [21]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from tqdm import tqdm

**selenium test**

In [35]:
target_url = 'https://search.naver.com/search.naver?sm=tab_sly.hst&where=nexearch&query=종로구%20통인동%20서촌소국밥'

driver = webdriver.Chrome('C:/worksplaces/workspace_project/project_visual/drivers/chromedriver.exe')

driver.get(target_url)
sleep(1)
#driver.find_element_by_xpath('//*[@id="_title"]/a').click()

C:\Users\wogml\AppData\Local\Temp/ipykernel_19932/1155029605.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/worksplaces/workspace_project/project_visual/drivers/chromedriver.exe')


**beautifulsoup test**

내일 하자 !!! 
메뉴 나누기

In [55]:
target_url = 'https://search.naver.com/search.naver?sm=tab_sly.hst&where=nexearch&query=종로구+창신동+실비집'

result = []
tmp = dict()

resp = requests.get(target_url)
soup = BeautifulSoup(resp.text, 'html.parser')

tmp['address'] = address_list[3]

if soup.find('span', {'class':'_3XamX'}):
    bistro_nm = soup.find('span', {'class':'_3XamX'}).text
    bistro_fd = soup.find('span', {'class':'_3ocDE'}).text
    bistro_tel = soup.find('span', {'class':'_3ZA0S'}).text

    tmp['storename'] = bistro_nm
    tmp['storetype'] = bistro_fd
    tmp['telephone'] = bistro_tel
    
elif soup.find('span', {'class':'place_bluelink _1RlVL'}):
    if len(soup.find_all('mark')) <= 2:
        
        bistro_nm1 = soup.find_all('mark')
        tmp['storename'] = bistro_nm1[0].text + bistro_nm1[1].text
        
    
    elif soup.find('mark'):
        bistro_nm2 = soup.find('mark')
        tmp['storename'] = bistro_nm2.text
    
    else:
        pass
    
    bistro_fd = soup.find('span', {'class':'_1tj6W'}).text
    tmp['storetype'] = bistro_fd

        
    

if soup.find('ul', {'class':'_1WK4L _2EEOB'}):
    
    menu_div1 = soup.find('ul', {'class':'_1WK4L _2EEOB'})
    menu_list1 = menu_div1.select('li')

    menu_lst1 = []

    for chd1 in menu_list1:

        menu_nm1 = chd1.find('a').string
        price1 = chd1.find('div', {'class':'_18uV7'}).text

        menu_lst1.append(menu_nm1 + " " + price1)

        tmp['menu'] = menu_lst1
    
    result.append(tmp)

    
elif soup.find('ul', {'class':'V1UmJ Zew_k'}):
    
    menu_div2 = soup.find('ul', {'class':'V1UmJ Zew_k'})
    menu_list2 = menu_div2.select('li')
    
    menu_lst2 = []
    
    for chd2 in menu_list2:
        menu_nm2 = chd2.find('a').text
        price2 = chd2.find('em').text
        
        menu_lst2.append(menu_nm2 + " " + price2)
        
        tmp['menu'] = menu_lst2
        
    result.append(tmp) 
    
elif soup.find('div', {'class':'_1mAZf'}):
    menu_span = soup.find_all('span', {'class':'_1TRlX'})
    
    menu_lst3 = []
    #print(menu_div3)
    
    tmp['telephone'] = menu_span[0].text
    
    menu_lst3.append(menu_span[-2].text)
    menu_lst3.append(menu_span[-1].text)
    
    tmp['menu'] = menu_lst3
    
    
    result.append(tmp)
        
    
        
        
        
print(result)

[{'address': '종로구 통인동 서촌소국밥', 'storename': '실비집', 'storetype': '전,빈대떡', 'telephone': '010-8835-5294', 'menu': ['모듬전(2인) 5,000', '김치찌게(2인) 5,000']}]


In [74]:
test_list = address_list[:100]

In [75]:
len(address_list)

124910

In [76]:
###백업용 

In [80]:
result = []

def open_browser(address_list):
    
    for i in tqdm(range(len(address_list) )):
        
        target_url = 'https://search.naver.com/search.naver?sm=tab_sly.hst&where=nexearch&query='

        url = target_url + address_list[i]
        
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        try:
            # storename
            bistro_nm = soup.find('span', {'class':'_3XamX'}).text
            # storetype
            bistro_fd = soup.find('span', {'class':'_3ocDE'}).text

            tmp = dict()
            tmp['storeaddr'] = address_list[i]
            tmp['storename'] = bistro_nm
            tmp['storetype'] = bistro_fd

        
            # 전화번호 
            try:
                bistro_tel = soup.find('span', {'class':'_3ZA0S'}).text
                tmp['callnum'] = bistro_tel

            except:
                tmp['callnum'] = ' '
        
        except: 
            pass

                
                
        if soup.find('ul', {'class':'_1WK4L _2EEOB'}):
            
            menu_div = soup.find('ul', {'class':'_1WK4L _2EEOB'})
            menu_list = menu_div.select('li')

            menu_lst = []

                # 메뉴
            for chd in menu_list:
                menu_nm = chd.find('a').text
                price = chd.find('div', {'class':'_18uV7'}).text

                menu_lst.append(menu_nm + " " + price)
                tmp['menu'] = menu_lst

            result.append(tmp)
                
        elif soup.find('ul', {'class':'V1UmJ Zew_k'}):
    
            menu_div2 = soup.find('ul', {'class':'V1UmJ Zew_k'})
            menu_list2 = menu_div2.select('li')

            menu_lst2 = []

            for chd2 in menu_list2:
                menu_nm2 = chd2.find('a').text
                price2 = chd2.find('em').text

                menu_lst2.append(menu_nm2 + " " + price2)

                tmp['menu'] = menu_lst2

            result.append(tmp) 
            
        else: 
            pass

In [ ]:
open_browser(address_list)

  4%|██▉                                                                      | 4977/124910 [45:07<19:17:45,  1.73it/s]

In [82]:
result

[{'storeaddr': '종로구 통인동 서촌소국밥',
  'storename': '서촌소국밥',
  'storetype': '한식',
  'callnum': '070-8885-7677',
  'menu': ['소머리국밥 12,000원', '제육볶음 10,000원', '사골만두국 10,000원']},
 {'storeaddr': '종로구 창신동 빽다방 동대문역점',
  'storename': '빽다방 동대문역점',
  'storetype': '테이크아웃커피',
  'callnum': '0507-1341-0950',
  'menu': ['아메리카노HOT 1,500원',
   '아메리카노ICE 2,000원',
   '더블에스프레소 1,500원',
   '원조커피 2,500원']},
 {'storeaddr': '종로구 소격동 주식회사 가배도 삼청',
  'storename': '가배도 삼청',
  'storetype': '카페',
  'callnum': '02-732-4542',
  'menu': ['말차 라떼 6,500원', '티라미수(일반/말차/얼그레이) 7,000원', '가배 4,900원']},
 {'storeaddr': '종로구 충신동 기생',
  'storename': '기생',
  'storetype': '우동,소바',
  'callnum': ' ',
  'menu': ['우동 12,000원', '고기채소나베 12,000원', '오뎅채소나베 12,000원']},
 {'storeaddr': '종로구 무악동 상구네백반',
  'storename': '상구네집',
  'storetype': '백반,가정식',
  'callnum': '0507-1325-5324',
  'menu': ['코다리찜 소 20,000원', '올갱이해장국 10,000원', '북어콩나물해장국 8,000원']},
 {'storeaddr': '종로구 연건동 서울대병원 델라코트',
  'storename': '델라코트 서울대병원점',
  'storetype': '푸드코트',
  'callnum'